### Imports

In [1]:
import scrapy
import pandas as pd
from scrapy.crawler import CrawlerProcess

### Global Object for Storing Data

In [2]:
data = {
    'Title': [],
    'Link': [],
    'Rating': [],
    'Ref': [],
    'Price': [],
    'MRP': [],
    'Images': [],
    'Description':[]

}

### HTTP Header for Crawling

In [3]:
h = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
    # Add More Headers here if required
}

### Spider Class

In [4]:
class NetShoes(scrapy.Spider):
    name = "netShoes"

    def start_requests(self):
        req = scrapy.Request(
            'https://www.netshoes.com.br/lst/top-marcas/adidas', headers=h)
        yield req

    def parse(self, response):
        data['Title'] = response.xpath(
            '//div[@class="wrapper"]/a/@title').getall()
        links = response.xpath('//div[@class="wrapper"]/a/@href').getall()
        for link in links:
            data['Link'].append("https:"+link)
        for link in data['Link']:
            yield scrapy.Request(url=link, headers=h, callback=self.parse2)

    def parse2(self, response):

        # XPATHS
        xRating = '//span[@class="rating-box__value"]/text()'
        xRef = '//p[@class="reference"]/span/text()'
        xPrice = '//div[@class="default-price"]//strong/text()'
        xMRP = '//del/text()'
        xDescTitle = '//section[@class="feature-values"]//li/strong/text()'
        xDescVal = '//section[@class="feature-values"]//li/text()'
        xImg = '//figure/img[@itemprop="image"]/@src'

        data['Rating'].append(response.xpath(xRating).get())
        data['Ref'].append(response.xpath(xRef).get())
        data['Price'].append(response.xpath(xPrice).get())
        data['MRP'].append(response.xpath(xMRP).get())


        desc = []
        desc_title = response.xpath(xDescTitle).getall()
        desc_val = response.xpath(xDescVal).getall()
        for i in range(len(desc_title)):
            txt = desc_title[i]+" "+desc_val[i]
            desc.append(txt)
        data['Description'].append(desc);

        img = []
        images = response.xpath(xImg).getall()
        for url in images:
            img.append(str.split(url,'?')[0])
        data['Images'].append(img)

### Start Crawler

In [5]:
process = CrawlerProcess()
process.crawl(NetShoes)
process.start()

2022-03-02 20:45:48 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-03-02 20:45:48 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.1.0, Python 3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 36.0.1, Platform Windows-10-10.0.22000-SP0
2022-03-02 20:45:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-03-02 20:45:48 [scrapy.crawler] INFO: Overridden settings:
{}
2022-03-02 20:45:48 [scrapy.extensions.telnet] INFO: Telnet Password: 3f226e29638a9df0
2022-03-02 20:45:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-03-02 20:45:48 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthM

### Display Data

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(data)
df

,Title,Link,Rating,Ref,Price,MRP,Images,Description
0,Tênis Adidas Coreracer Masculino,https://www.netshoes.com.br/tenis-adidas-coreracer-masculino-preto+branco-NQQ-4635-205,4.56,NQQ-4635-205-40,"R$ 179,99","R$ 249,99","[https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom1.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom2.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom3.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom4.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom5.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-coreracer-masculino/05/NQQ-4635-205/NQQ-4635-205_zoom6.jpg]","[Indicado para: Caminhada, Corrida, Material: Têxtil, Composição: Cabedal: Têxtil e Mesh; Entressola: EVA; Solado: Borracha e contra-salto de inspiração tecnológica, Pisada: Neutra, Garantia do Fabricante: Contra defeito de fabricação, Origem: Estrangeira - Adquirida no mercado interno]"
1,Tênis Adidas Coreracer Feminino,https://www.netshoes.com.br/tenis-adidas-coreracer-feminino-preto-NQQ-4636-793,4.45,3ZP-1049-028-38,"R$ 339,99","R$ 499,99","[https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom1.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom2.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom3.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom4.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom5.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom6.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom7.jpg, https://static.netshoes.com.br/produtos/tenis-adidas-kaptir-20-masculino/28/3ZP-1049-028/3ZP-1049-028_zoom8.jpg]","[Departamento BS: Esporte, Indicado para: Dia a Dia, Estilo da Peça: Com logo, Material: Malha, Material Interno: Têxtil, Altura do Cano: Cano Baixo]"
2,Tênis Adidas Coreracer Masculino,https://www.netshoes.com.br/tenis-adidas-coreracer-masculino-azul+petroleo+branco-NQQ-4635-012,None,3ZP-2540-042-01,"R$ 69,99","R$ 99,99",[https://static.netshoes.com.br/produtos/overgrip-adidas-c-3-unidades/42/3ZP-2540-042/3ZP-2540-042_zoom1.jpg],"[Indicado para: Dia a Dia, Quantidade: 03 unidades, Composição: Poliuretano, Espessura: 100cm X 2,5cm X 0,75mm, Garantia do Fabricante: Contra defeito de fabricação, Origem: Estrangeira - Adquirida no mercado interno]"
3,Tênis Adidas Runfalcon 2.0 Masculino,https://www.netshoes.com.br/tenis-adidas-runfalcon-20-masculino-preto+branco-3ZP-0573-026,None,3ZP-2540-014-01,"R$ 74,99","R$ 99,99",[https://static.netshoes.com.br/produtos/overgrip-adidas-c-3-unidades/14/3ZP-2540-014/3ZP-2540-014_zoom1.jpg],"[Indicado para: Dia a Dia, Quantidade: 03 unidades, Composição: Poliuretano, Espessura: 100cm X 2,5cm X 0,75mm, Garantia do Fabricante: Contra defeito de fabricação, Origem: Estrangeira - Adquirida no mercado interno]"
4,Tênis Adidas Runfalcon 2.0 Masculino,https://www.netshoes.com.br/tenis-adidas-runfalcon-20-masculino-preto+cinza-3ZP-0573-172,None,3ZP-2540-030-01,"R$ 74,99","R$ 99,99",[https://static.netshoes.com.br/produtos/overgrip-adidas-c-3-unidades/30/3ZP-2540-030/3ZP-2540-030_zoom1.jpg],"[Indicado para: Dia a Dia, Quantidade: 03 unidades, Composição: Poliuretano, Espessura: 100cm X 2,5cm X 0,75mm, Garantia do Fabricante: Contra defeito de fabricação, Origem: Estrangeira - Adquirida no mercado interno]"
5,Tênis Adidas Terrex Soulstride Trail Outdoor Masculino,https://www.netshoes.com.br/te